In [1]:
import os
import sys
import re
import time

from datetime import datetime
from tqdm import tqdm_notebook

import pandas as pd

import warnings
warnings.filterwarnings(action = 'ignore')

from selenium import webdriver
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

---

# 1. 연도별 데이터 수집

In [2]:
# 추출한 데이터를 사용자가 지정한 경로와 파일명으로 저장해주는 함수 생성
def data_save(gwang_data = None, gwanhal_data = None, PATH = None, gwang_file_name = None, gwanhal_file_name = None):
    
    '''
    Function Description
    --------------------------------------------------------------------------------------------------------
    gwang_data : pd.DataFrame 형태의 광역시도 데이터를 입력해주면 됩니다.
    
    gwanhal_data : pd.DataFrame 형태의 관할서 데이터를 입력해주면 됩니다.
    
    PATH : 데이터를 저장할 경로를 입력해주면 됩니다.(ex: './data')
    
    gwang_file_name : 저장할 광역시도 데이터의 이름을 확장자명을 포함해 입력해주면 됩니다.(ex: '광역시도데이터.csv')
    
    gwanhal_file_name : 저장할 관할서 데이터의 이름을 확장자명을 포함해 입력해주면 됩니다.(ex: '관할서데이터.csv')
    '''
    gwang_data.to_csv(PATH + '/' + gwang_file_name, encoding = 'CP949', index = False)
    gwanhal_data.to_csv(PATH + '/' + gwanhal_file_name, encoding = 'CP949', index = False)

# 연안체험활동현황의 '연안체험활동통계' 정보를 크롤링 해주는 함수 생성
def coastal_activity_crawling(start_date = None, end_date = None):
    
    '''
    Function Description
    --------------------------------------------------------------------------------------------------------
    start_date : 데이터를 수집할 시작 일자를 지정해주면 됩니다.(ex: '2017-01-01')
    
    end_date : 데이터를 수집할 마지막 일자를 지정해주면 됩니다.(ex: '2017-12-31')
    
    Caution
    --------------------------------------------------------------------------------------------------------
    chromedriver.exe 파일은 현재 파이썬 코드가 실행되는 위치에 저장이 되어있어야 합니다.
    
    [버전 일치 문제 발생시 참고 사이트]
    https://blog.naver.com/kiddwannabe/221539689821
    '''
    # Selenium 옵션 생성
    options = webdriver.ChromeOptions()
    
    # 크롬 창을 생성하지 않도 수행할 수 있는 옵션 지정
    options.add_argument('headless')
    
    # Chrome Driver 파일이 현재 작업하고 있는 경로에 저장되어 있다는 가정 하에 수행
    driver = webdriver.Chrome(executable_path = './chromedriver.exe', options = options)
    
    # 연안체험활동현황 홈페이지
    driver.get('https://imsm.kcg.go.kr/CSMS/main/sta/StaSttusRB.do')
    time.sleep(1)
    
    #---------------------------------------------------------------------------------------------------------------
    ## 광역시도별 데이터 수집
    #---------------------------------------------------------------------------------------------------------------
    select = Select(driver.find_element_by_id('searchCondition'))
    select.select_by_visible_text('광역시도별')
    
    # 일자 선택
    driver.find_element_by_id('searchKeywordFrom').send_keys(start_date)
    driver.find_element_by_id('searchKeywordTo').send_keys(end_date)
    
    # '검색' 버튼 클릭
    element = driver.find_element_by_xpath('//*[@id="staSttus"]/section/div[2]/table/tbody/tr[2]/td/div/button')
    element.click()
    time.sleep(1)
    
    # 화면에 출력된 페이지 소스 저장
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    # 변수명과 값을 저장하기 위한 리스트 객체 생성
    columns_list = []
    value_list = []
    
    # 추출하고자 하는 정보는 <tbody id='acdntList'> 태그의 아래 <tr> 태그에 포함되어 있다.
    for index, tag in enumerate(soup.select('#acdntList > tr')):
        
        # 인덱스가 0인 경우에는 변수명에 해당된다.
        if index == 0:
            for td_tag in tag.find_all('td'):
                columns_list.append(td_tag.get_text())
        
        # 그 외의 경우에는 값에 해당된다.
        else:
            value_list.append([td_tag.get_text().strip() for td_tag in tag.find_all('td')])
    
    # 광역시도 데이터 생성
    gwang_data = pd.DataFrame(data = value_list, columns = columns_list)
    
    #---------------------------------------------------------------------------------------------------------------
    ## 관할서별 데이터 수집
    #---------------------------------------------------------------------------------------------------------------
    select = Select(driver.find_element_by_id('searchCondition'))
    select.select_by_visible_text('관할서별')
    
    # 일자 선택
    driver.find_element_by_id('searchKeywordFrom').send_keys(start_date)
    driver.find_element_by_id('searchKeywordTo').send_keys(end_date)
    
    # '검색' 버튼 클릭
    element = driver.find_element_by_xpath('//*[@id="staSttus"]/section/div[2]/table/tbody/tr[2]/td/div/button')
    element.click()
    time.sleep(1)
    
    # 화면에 출력된 페이지 소스 저장
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    # 변수명과 값을 저장하기 위한 리스트 객체 생성
    columns_list = []
    value_list = []
    
    # 추출하고자 하는 정보는 <tbody id='acdntList'> 태그의 아래 <tr> 태그에 포함되어 있다.
    for index, tag in enumerate(soup.select('#acdntList > tr')):
        
        # 인덱스가 0인 경우에는 변수명에 해당된다.
        if index == 0:
            for td_tag in tag.find_all('td'):
                columns_list.append(td_tag.get_text())
        
        # 그 외의 경우에는 값에 해당된다.
        else:
            value_list.append([td_tag.get_text().strip() for td_tag in tag.find_all('td')])
    
    # 관할서 데이터 생성
    gwanhal_data = pd.DataFrame(data = value_list, columns = columns_list)
    
    # 모든 데이터가 생성되었으므로 Chrome Driver 종료
    driver.close()
    
    #---------------------------------------------------------------------------------------------------------------
    # 생성한 데이터 저장하기
    #---------------------------------------------------------------------------------------------------------------
    PATH = input('파일을 저장할 경로를 입력해주세요.(ex: ./공공데이터/data) :'); print()
    gwang_file_name = input('저장할 광역시도 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) :'); print()
    gwanhal_file_name = input('저장할 관할서 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) :')
    
    data_save(gwang_data = gwang_data, gwanhal_data = gwanhal_data, PATH = PATH, gwang_file_name = gwang_file_name, gwanhal_file_name = gwanhal_file_name)

In [6]:
coastal_activity_crawling(start_date = '2017-01-01', end_date = '2017-12-31')

파일을 저장할 경로를 입력해주세요.(ex: ./공공데이터/data) : C:/Users/KCG/Desktop/새 폴더


저장할 광역시도 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) : 2017년 해양경찰청_광역시도별 연안체험활동 활동인원 통계.csv


저장할 관할서 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) : 2017년 해양경찰청_관할서별 연안체험활동 활동인원 통계.csv


In [3]:
coastal_activity_crawling(start_date = '2021-01-01', end_date = '2021-12-31')

파일을 저장할 경로를 입력해주세요.(ex: ./공공데이터/data) : C:/Users/KCG/Desktop/KCG/Crawling/SaveData


저장할 광역시도 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) : 2021년 해양경찰청_광역시도별 연안체험활동 활동인원 통계.csv


저장할 관할서 데이터의 이름을 확장자명을 포함해 입력해주세요.(ex: 저장데이터.csv) : 2021년 해양경찰청_관할서별 연안체험활동 활동인원 통계.csv


---

# 2. 일별 데이터 수집

In [2]:
# 일별로 데이터 수집을 수행하고 수집한 데이터를 저장해주는 함수 생성
def daily_coastal_activity_crawling(type_name = None, start_date = None, end_date = None, PATH = None, file_name = None):

    '''
    Function Description
    --------------------------------------------------------------------------------------------------------
    type_name : '광역시도별', '관할서별' 중에서 하나를 입력해주면 됩니다.
    
    start_date : 데이터를 수집할 시작 일자를 지정해주면 됩니다.(ex: '2017-01-01')
    
    end_date : 데이터를 수집할 마지막 일자를 지정해주면 됩니다.(ex: '2017-12-31')
    
    PATH : 데이터를 저장할 경로를 입력해주면 됩니다.(ex: './data')
    
    file_name : 저장할 데이터의 이름을 확장자명을 포함해 입력해주면 됩니다.(ex: '일별 데이터.csv')

    Caution
    --------------------------------------------------------------------------------------------------------
    chromedriver.exe 파일은 현재 파이썬 코드가 실행되는 위치에 저장이 되어있어야 합니다.
    
    [버전 일치 문제 발생시 참고 사이트]
    https://blog.naver.com/kiddwannabe/221539689821
    '''
    #---------------------------------------------------------------------------------------------------------------
    # Selenium 실행 과정
    #---------------------------------------------------------------------------------------------------------------
    
    # Selenium 옵션 생성
    options = webdriver.ChromeOptions()
    
    # 크롬 창을 생성하지 않고 수행할 수 있는 옵션 지정
    options.add_argument('headless')
    
    # Chrome Driver 수행
    driver = webdriver.Chrome(executable_path = './chromedriver.exe', options = options)
    
    # 연안체험활동 종합정보 - 연안안전정보 - 연안사고통계 홈페이지
    driver.get('https://imsm.kcg.go.kr/CSMS/main/sta/StaSttusRB.do')
    time.sleep(1)
    
    #---------------------------------------------------------------------------------------------------------------
    # 데이터 수집 과정
    #---------------------------------------------------------------------------------------------------------------
    try:
        # '통계 유형' 선택
        select = Select(driver.find_element_by_id('searchCondition'))
        select.select_by_visible_text('{}'.format(type_name))

        # 사용자가 지정한 시작날짜와 마지막날짜까지의 기간을 '%Y-%m-%d' 타입의 문자열로 저장
        date_list = [date.strftime('%Y-%m-%d') for date in pd.date_range(start = start_date, end = end_date).tolist()]
        print('총 데이터 수집 건 수:', len(date_list))

        # 최종 데이터를 반환하기 위한 DataFrame 객체 생성
        return_data = pd.DataFrame()

        # 변수명을 저장하기 위한 리스트 객체 생성
        column_list = []

        # '일자'를 변경해가면서 데이터 수집
        for first_index, date in tqdm_notebook(enumerate(date_list), desc = '데이터 수집 과정 진행 상태'):

            # '일자(발생일 기준)' 선택하기 위해 처음 위치와 마지막 위치를 객체로 저장
            start_box = driver.find_element_by_id('searchKeywordFrom')
            end_box = driver.find_element_by_id('searchKeywordTo')
            
            # 값 입력
            start_box.send_keys(date)
            end_box.send_keys(date)
            
            # '검색' 버튼 클릭
            element = driver.find_element_by_xpath('//*[@id="staSttus"]/section/div[2]/table/tbody/tr[2]/td/div/button')
            element.click()
            time.sleep(1)

            # 화면에 출력된 페이지 소스 저장
            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')

            # 추출하고자 하는 정보들은 <tbody id="acdntList"> 태그의 아래 <tr> 태그에 포함되어 있다.
            # <tr> 태그 중에서 class 속성을 가지고 있는 <tr class="aliceBlue"> 태그는 값들의 '변수명'을 나타낸다.
            # 변수명은 한 번만 저장하면 되므로 처음 검색하는 데이터에서만 추출하도록 하자.

            if first_index == 0:
                for column_value in [td_tag.get_text().strip() for td_tag in soup.find('tr', class_ = 'aliceBlue').find_all('td')]:
                    column_list.append(column_value)

            value_list = []

            # 다음으로 변수명에 해당되는 값들을 추출하도록 하자.
            for second_index, tag in enumerate(soup.select('#acdntList > tr')):

                # 인덱스가 0인 경우에는 변수명에 해당되므로 해당 값을 제외하고 추출하도록 한다.
                if second_index != 0:
                    value_list.append([td_tag.get_text().strip() for td_tag in tag.find_all('td')])

            # 일별로 수집한 데이터를 DataFrame으로 저장
            daily_data = pd.DataFrame(data = value_list, columns = column_list).drop('활동인원', axis = 1)

            # '날짜' 변수를 추가
            daily_data['날짜'] = date

            # 일자를 변경해가면서 반환할 데이터에 병합
            return_data = return_data.append(daily_data)
            
            # 해당 날자의 데이터 수집을 마친 후, '일자(발생일 기준)' 부분에 입력된 값을 지워줘야 한다.
            start_box.clear()
            end_box.clear()
            time.sleep(0.5)

        # '날짜' 변수를 맨 앞으로 지정해주기
        columns = return_data.columns.tolist()
        tmp = columns[-1]
        del columns[-1]
        columns.insert(0, tmp)

        return_data = return_data[columns]
        return_data.reset_index(drop = True, inplace = True)

        # Chrome Driver 종료
        driver.close()
    
    # 데이터 수집 과정에서 오류가 발생하는 경우 Chrome Driver 종료
    except:
        driver.close()
    
    #---------------------------------------------------------------------------------------------------------------
    # 데이터 저장 과정
    #---------------------------------------------------------------------------------------------------------------
    return_data.to_csv(PATH + '/' + file_name, encoding = 'CP949', index = False)
    
    return return_data

In [3]:
data = daily_coastal_activity_crawling(type_name = '광역시도별', start_date = '2010-01-01', end_date = '2021-09-30',
                                       PATH = 'C:/Users/KCG/Desktop/[목록등록관리]_업데이트 데이터/1. 연안사고 및 연안체험활동/Daily Data Crawling',
                                       file_name = '해양경찰청_광역시도별 연안체험활동 활동인원 상세.csv')

총 데이터 수집 건 수: 4291


데이터 수집 과정 진행 상태: 0it [00:00, ?it/s]